In [1]:
import json
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import os
import numpy as np
from time import time
import os
from collections import OrderedDict
from tqdm import tqdm, trange
import torch.optim as optim
import random
import torch.nn.functional as F

torch.set_num_threads(1)

def seed_everything(seed):
    """
    Changes the seed for reproducibility. 
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(1024)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path = r"adult.json"
test_path = r"adult_test.json"
train_df = pd.read_json(path)
test_df = pd.read_json(test_path)
test_df

,age,fnlwgt,education-num,marital-status,race,sex,capital-gain,capital-loss,hours-per-week,native-country,...,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,occupation_10,occupation_11,occupation_12,occupation_13
0,-1.029005,0.352098,-1.216563,1,1,1,0,0,-0.078889,1,...,0,0,0,0,1,0,0,0,0,0
1,-0.057423,-0.944995,-0.434898,0,0,1,0,0,0.750131,1,...,0,0,0,1,0,0,0,0,0,0
2,-0.804794,1.395062,0.737599,0,0,1,0,0,-0.078889,1,...,0,0,0,0,0,0,0,1,0,0
3,0.391000,-0.277369,-0.044066,0,1,1,1,0,-0.078889,1,...,0,0,0,0,1,0,0,0,0,0
5,-0.356371,0.085944,-1.607395,1,0,1,0,0,-0.907909,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16275,-0.431108,0.526407,1.128431,1,0,1,0,0,-0.078889,1,...,0,0,0,0,0,0,0,0,0,0
16276,0.017315,0.244316,1.128431,1,0,0,0,0,-0.410497,1,...,0,0,0,0,0,0,0,0,0,0
16278,-0.057423,1.755175,1.128431,0,0,1,0,0,0.750131,1,...,0,0,0,0,0,0,0,0,0,0
16279,0.391000,-1.001078,1.128431,1,1,1,1,0,-0.078889,1,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from sklearn.model_selection import train_test_split

def get_naive_dataset(dataset):
    X = dataset.drop(['income'], axis=1)
    y = dataset['income']
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 2**28 )# 2**(0))
    a_train = x_train['sex']
    a_test = x_test['sex']
    x_train = x_train.drop(['race','sex'], axis=1)
    x_test = x_test.drop(['race','sex'], axis=1)
    return (x_train, y_train, a_train), (x_test, y_test, a_test)

(x_train, y_train, a_train), (x_valid, y_valid, a_valid) = get_naive_dataset(train_df)
x_test, y_test, a_test = test_df.drop(['income','race','sex'], axis=1), test_df['income'], test_df['sex']


batch_size = 256
def make_dataloader(data, y, a,batch_size):
    dataset = BasicDataset(data, y, a)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader


class BasicDataset(torch.utils.data.Dataset):
    def __init__(self, data, target, sensitive):
        super().__init__()
        self.data = torch.tensor(data.values)
        self.target = torch.tensor(target.values)
        self.sensitive = torch.tensor(sensitive.values)

    def __len__(self):
        return self.data.size(0)
    
    def __getitem__(self, idx):
        return self.data.float()[idx], self.target[idx], self.sensitive[idx]
    
training_loader = make_dataloader(x_train, y_train, a_train, batch_size)
valid_loader = make_dataloader(x_valid, y_valid, a_valid, batch_size)
test_loader = make_dataloader(x_test, y_test, a_test, batch_size)

In [3]:
class Tablur_Model(nn.Module):
    
    def __init__(self):
        super(Tablur_Model, self).__init__()
        nodes = 32
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(32, nodes)
        self.fc2 = nn.Linear(nodes, 2*nodes)
        

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return x

In [4]:
from sklearn.metrics import confusion_matrix, accuracy_score



def test(net, classifier, dataloader, print_fairness=True):
    net.eval()
    test_pred = []
    test_gt = []
    sense_gt = []
    female_predic = []
    female_gt = []
    male_predic = []
    male_gt = []
    correct_00, total_00 = 0, 0
    correct_01, total_01 = 0, 0
    correct_10, total_10 = 0, 0
    correct_11, total_11 = 0, 0
    classifier = classifier.to(device)
    with torch.no_grad():
        with tqdm(dataloader, unit="batch") as tepoch:
            for content in tepoch:
                test, label, sensitive = content
                test = test.to(device)
                
                mask_00 = ((label == 0) & (sensitive == 0))
                mask_01 = ((label == 0) & (sensitive == 1))
                mask_10 = ((label == 1) & (sensitive == 0))
                mask_11 = ((label == 1) & (sensitive == 1))
                test_feature = net(test)
                prediction = classifier(test_feature)
                _, predic = torch.max(prediction.data, 1)
                predic = predic.detach().cpu()
                test_pred.extend(predic.numpy())
                test_gt.extend(label)
                sense_gt.extend(sensitive)
                correct_00 += (predic[mask_00] == label[mask_00]).float().sum().item()
                total_00 += mask_00.float().sum().item()

                correct_01 += (predic[mask_01] == label[mask_01]).float().sum().item()
                total_01 += mask_01.float().sum().item()

                correct_10 += (predic[mask_10] == label[mask_10]).float().sum().item()
                total_10 += mask_10.float().sum().item()

                correct_11 += (predic[mask_11] == label[mask_11]).float().sum().item()
                total_11 += mask_11.float().sum().item() 
                
        acc_00 = correct_00 / total_00
        acc_01 = correct_01 / total_01
        acc_10 = correct_10 / total_10
        acc_11 = correct_11 / total_11

        print(f'Accuracy for y=0, s=0: {acc_00}', total_00)
        print(f'Accuracy for y=0, s=1: {acc_01}', total_01)
        print(f'Accuracy for y=1, s=0: {acc_10}', total_10)
        print(f'Accuracy for y=1, s=1: {acc_11}', total_11)  
        wga = min(acc_00, acc_01, acc_10, acc_11)


        for i in range(len(sense_gt)):
            if sense_gt[i] == 0:
                female_predic.append(test_pred[i])
                female_gt.append(test_gt[i])
            else:
                male_predic.append(test_pred[i])
                male_gt.append(test_gt[i])

        female_CM = confusion_matrix(female_gt, female_predic)    
        male_CM = confusion_matrix(male_gt, male_predic) 
        female_dp = (female_CM[1][1]+female_CM[0][1])/(female_CM[0][0]+female_CM[0][1]+female_CM[1][0]+female_CM[1][1])
        male_dp = (male_CM[1][1]+male_CM[0][1])/(male_CM[0][0]+male_CM[0][1]+male_CM[1][0]+male_CM[1][1])
        female_TPR = female_CM[1][1]/(female_CM[1][1]+female_CM[1][0])
        male_TPR = male_CM[1][1]/(male_CM[1][1]+male_CM[1][0])
        female_FPR = female_CM[0][1]/(female_CM[0][1]+female_CM[0][0])
        male_FPR = male_CM[0][1]/(male_CM[0][1]+male_CM[0][0])
        if print_fairness == True:
            EOd = 0.5*(abs(female_FPR-male_FPR)+abs(female_TPR-male_TPR))
            print('DP',abs(female_dp - male_dp))
            print('EOP', abs(female_TPR - male_TPR))
            print('EoD', EOd)
            print('acc', accuracy_score(test_gt, test_pred))
        else:
            EOd = 0.5*(abs(female_FPR-male_FPR)+abs(female_TPR-male_TPR))
        return accuracy_score(test_gt, test_pred), EOd, wga


def train_model():
    model = Tablur_Model()
    model = model.to(device)
    epoch = 100
    criterion = nn.CrossEntropyLoss()
    mean_criterion = nn.MSELoss()
    acc = 0
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    train_loss = []
    valid_loss =[]
    wga = 0
    classifier = nn.Linear(2*32, 2)
        
    for epoches in range(epoch):
        with tqdm(training_loader, unit="batch") as tepoch:
            model.train()
            running_loss = 0.0
            feature_y_0_a0 = []
            feature_y_0_a1 = []
            feature_y_1_a0 = []
            feature_y_1_a1 = []
            loss00 = 0
            loss01 = 0
            loss10 = 0
            loss11 = 0
            
            with torch.no_grad(): 
                for step, (valid_input, valid_target, validsensitive) in enumerate(valid_loader):
                    valid_input = valid_input.to(device)
                    with torch.no_grad():
                        valid_feature = model(valid_input)
                        label = valid_target.squeeze().detach().cpu()
                        mask_00 = ((label == 0) & (validsensitive == 0))
                        mask_01 = ((label == 0) & (validsensitive == 1))
                        mask_10 = ((label == 1) & (validsensitive == 0))
                        mask_11 = ((label == 1) & (validsensitive == 1))
                        g1 = valid_feature[mask_00]
                        g2 = valid_feature[mask_01]
                        g3 = valid_feature[mask_10]
                        g4 = valid_feature[mask_11]
                        feature_y_0_a0.extend(g1.detach().cpu().numpy())
                        feature_y_0_a1.extend(g2.detach().cpu().numpy())
                        feature_y_1_a0.extend(g3.detach().cpu().numpy())
                        feature_y_1_a1.extend(g4.detach().cpu().numpy())

                feature_g2 = np.array(feature_y_0_a1)
                feature_g4 = np.array(feature_y_1_a1)
                feature_g2_tensor = torch.from_numpy(feature_g2)
                feature_g4_tensor = torch.from_numpy(feature_g4)

                mu_1 = torch.mean(feature_g2_tensor, 0)
                mu_1 = mu_1 /torch.norm(mu_1)
                mu_2 = torch.mean(feature_g4_tensor, 0)
                mu_2 = mu_2 /torch.norm(mu_2)
                weight = torch.cat((mu_1.unsqueeze(0), mu_2.unsqueeze(0)), 0)
                #print("sim:",  F.cosine_similarity(mu_1.unsqueeze(0), mu_2.unsqueeze(0)) )

                with torch.no_grad():
                    classifier.weight = nn.Parameter(weight)
            
            
            
            for train_input, train_target, sensitive in tepoch:
                train_input = train_input.to(device)
                label = train_target.detach().cpu() 
                one_hot_labels = F.one_hot(train_target, num_classes=2)
                train_target = one_hot_labels.float().to(device)
                
                feature = model(train_input)
                classifier = classifier.to(device)
                outputs  = classifier(feature)
            
                mask_00 = ((label== 0) & (sensitive == 0))
                mask_01 = ((label == 0) & (sensitive == 1))
                mask_10 = ((label == 1) & (sensitive == 0))
                mask_11 = ((label == 1) & (sensitive == 1))
                
                count_00 = mask_00.sum()
                count_01 = mask_01.sum()
                count_10 = mask_10.sum()
                count_11 = mask_11.sum()
                
                if count_01==0 or count_10==0 or count_00 == 0 or count_11 == 0:
                    continue
                g1_f = feature[mask_00]
                g2_f = feature[mask_01]
                
                mu1 = torch.mean(g1_f, 0)
                mu2 = torch.mean(g2_f, 0)
                    
                g3_f = feature[mask_10]
                g4_f = feature[mask_11]
                
                mu3 = torch.mean(g3_f, 0)
                mu4 = torch.mean(g4_f, 0)
                
                loss_mean = mean_criterion(mu3, mu4) + mean_criterion(mu1, mu2)
                
                
                if count_00 > 0:
                    loss_00 = criterion(outputs[mask_00], train_target[mask_00])
                    loss00 += loss_00.item()
                else:
                    loss_00 = torch.tensor(0)
                if count_01 > 0:
                    loss_01 = criterion(outputs[mask_01], train_target[mask_01])
                    loss01 += loss_01.item()
                else:
                    loss_01 = torch.tensor(0)
                if count_10 > 0:
                    loss_10 = criterion(outputs[mask_10], train_target[mask_10])
                    loss10 += loss_10.item()
                else:
                    loss_10 = torch.tensor(0)
                if count_11 > 0:
                    loss_11 = criterion(outputs[mask_11], train_target[mask_11])
                    loss11 += loss_11.item()
                else:
                    loss_11 = torch.tensor(0)

                loss = loss_00 + loss_01 + loss_10 + loss_11 + loss_mean
                
                tepoch.set_postfix(ut_loss = loss.item())

                optimizer.zero_grad()    
                loss.backward()
                optimizer.step()
                tepoch.set_description(f"epoch %2f " % epoches)
        print('Testing')        
        _,  _ , wg = test(model, classifier, test_loader, False)
        if wg > wga:
            torch.save(model, 'Adult_model.pth')
            torch.save(classifier, 'classifier.pth')
            wga = wg

        

train_model()
print('************Evaluation************')
model = Tablur_Model()
classifier = nn.Linear(2*32, 2)
model = torch.load('Adult_model.pth')
model.eval()
classifier = torch.load('classifier.pth')
classifier.eval()
_,  _ , _ = test(model, classifier, test_loader, True)


epoch 0.000000 : 100%|█████████████████████████████████████████████████| 89/89 [00:30<00:00,  2.93batch/s, ut_loss=1.88]


Testing


100%|████████████████████████████████████████████████████████████████████████████████| 59/59 [00:11<00:00,  5.01batch/s]


Accuracy for y=0, s=0: 0.8560606060606061 4356.0
Accuracy for y=0, s=1: 0.7364363221016562 7004.0
Accuracy for y=1, s=0: 0.7827648114901257 557.0
Accuracy for y=1, s=1: 0.8017817371937639 3143.0


epoch 1.000000 : 100%|█████████████████████████████████████████████████| 89/89 [00:32<00:00,  2.74batch/s, ut_loss=1.46]


...

Testing


100%|████████████████████████████████████████████████████████████████████████████████| 59/59 [00:13<00:00,  4.33batch/s]


Accuracy for y=0, s=0: 0.8601928374655647 4356.0
Accuracy for y=0, s=1: 0.7602798400913764 7004.0
Accuracy for y=1, s=0: 0.822262118491921 557.0
Accuracy for y=1, s=1: 0.8027362392618518 3143.0
************Evaluation************


100%|████████████████████████████████████████████████████████████████████████████████| 59/59 [00:12<00:00,  4.69batch/s]


Accuracy for y=0, s=0: 0.8778696051423324 4356.0
Accuracy for y=0, s=1: 0.7824100513992005 7004.0
Accuracy for y=1, s=0: 0.8043087971274686 557.0
Accuracy for y=1, s=1: 0.7836461979000955 3143.0
DP 0.19345322517390454
EOP 0.020662599227373124
EoD 0.05806107648525255
acc 0.8110889774236387
